## 1 - Introduction
Data Storytelling 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import time
from scipy.stats import shapiro
import statsmodels.api as sm
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [60]:
# read data
start = time.time()
path = '/Users/cyuancheng/Documents/course/Springboard/Capstone1/data/'
df = pd.read_csv(path+'data_col_pickup.csv', low_memory=True, index_col=0, \
                 parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'issue_year'] )
end = time.time()
print('It takes  ' + str(end-start) +' sec to read the data')

It takes  51.631893157958984 sec to read the data


## 2 - PreProcessing

The dataset was download from [Lending Club](https://www.lendingclub.com/statistics/additional-statistics?). It contains information from 2015 to 2019. 

**Data manipulation**: 

    - Deal with missing value 
    - check each feature
        - observe distribution (normality for particular features)
        - Remove outliner (standard deviation)



In [61]:
df.head()

,issue_d,earliest_cr_line,last_pymnt_d,issue_year,int_rate,revol_util,term,grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,loan_amnt,funded_amnt,installment,annual_inc,int_rate.1,dti,revol_bal,delinq_2yrs,open_acc,pub_rec,fico_range_high,fico_range_low,total_pymnt,recoveries
0,2015-12-01,2005-01-01,2018-08-01,2015-01-01,13.99,87.4,60 months,C,< 1 year,RENT,Source Verified,Fully Paid,credit_card,OH,21000.0,21000.0,488.53,52000.0,13.99,14.47,20374.0,0.0,5.0,0.0,684.0,680.0,27285.807861,0.0
1,2015-12-01,1997-06-01,2020-01-01,2015-01-01,12.88,24.1,60 months,C,10+ years,MORTGAGE,Not Verified,Current,debt_consolidation,MO,16800.0,16800.0,381.23,118000.0,12.88,34.29,7849.0,0.0,11.0,1.0,684.0,680.0,18650.220000,0.0
2,2015-12-01,1995-04-01,2020-01-01,2015-01-01,12.88,32.7,60 months,C,6 years,MORTGAGE,Not Verified,Current,debt_consolidation,SC,22400.0,22400.0,508.30,95000.0,12.88,22.98,13028.0,0.0,16.0,0.0,714.0,710.0,24866.630000,0.0
3,2015-12-01,2002-04-01,2018-08-01,2015-01-01,6.49,34.5,36 months,A,6 years,RENT,Not Verified,Fully Paid,credit_card,PA,10000.0,10000.0,306.45,85000.0,6.49,13.07,10464.0,0.0,14.0,1.0,689.0,685.0,10998.971575,0.0
4,2015-12-01,1994-11-01,2017-04-01,2015-01-01,11.48,39.1,36 months,B,10+ years,MORTGAGE,Not Verified,Fully Paid,credit_card,RI,8000.0,8000.0,263.74,42000.0,11.48,34.80,7034.0,0.0,8.0,0.0,704.0,700.0,8939.580503,0.0
